# First step!
Remove adapter, kraken, assembly

# Remove adapter
using fastp

In [ ]:
#fastp 0.12.4

mkdir 01_adapter_removed

parallel --xapply -j 8 \
    'fastp \
        -i {1} \
        -o ./01_adapter_removed/{1/} \
        -I {2} \
        -O ./01_adapter_removed/{2/}' \
::: ./00_fastq/*_1.fastq.gz ::: ./00_fastq/*_2.fastq.gz


# Try Kraken on all samples

In [ ]:
mkdir 91_kraken

for sample in $(find ./01_adapter_removed/ -name "*_1.fastq.gz" -type f); do
    file_name=$(basename $sample)
    sample_name=${file_name%_*}
    echo $sample_name

    kraken2 --db /data3/databases/kranken2_database --threads 32 \
        --output ./91_kraken/${sample_name}_output.txt \
        --report ./91_kraken/${sample_name}_report.txt \
        --paired \
        ./01_adapter_removed/${sample_name}_1.fastq.gz ./01_adapter_removed/${sample_name}_2.fastq.gz 
    
done


# Try metaphlan on samples 7, 11, 13


In [ ]:
# start docker container
docker run -it --rm -v /data3/WINONA3/:/data3/WINONA3/ -w /data3/WINONA3/ biobakery/metaphlan:4.0.2 bash

for sample in $(find ./01_adapter_removed/ -name "*_1.fastq.gz" -type f); do
    file_name=$(basename $sample)
    sample_name=${file_name%_*}

    zcat 01_adapter_removed/${sample_name}_1.fastq.gz 01_adapter_removed/${sample_name}_2.fastq.gz > 01_cat/${sample_name}_cat.fastq

    metaphlan 01_cat/${sample_name}_cat.fastq \
        --input_type fastq \
        --bowtie2db /data3/WINONA3/db/metaphlan_202403/ --index mpa_vJun23_CHOCOPhlAnSGB_202403 \
        --bowtie2out 92_metaphlan4/${sample_name}.bowtie2out.bz2 \
        -o 92_metaphlan4/${sample_name}.profile.txt \
        --nproc 32
done



# Assembly
with metaspades

In [ ]:
# Assembly
mkdir 02_assembly

conda activate python3.9

for sample_name in $(cat sample_for_binning.txt); do
    echo $sample_name

    spades.py -o ./02_spades/02_assembly/$sample_name \
        --meta -t 64 -m 600 \
        -1 ./01_adapter_removed/${sample_name}_1.fastq.gz -2 ./01_adapter_removed/${sample_name}_2.fastq.gz 

    # remove all sequences below 150bp because how can the assembled be shorter than the reads?
    # seqkit 2.9.0, cite paper
    #seqkit seq 02_assembly/$sample_name/contigs.fasta -m 150 > 02_assembly/$sample_name/contigs_filt150.fasta

done

conda deactivate
conda activate seqkit

for sample_name in $(cat sample_for_binning.txt); do
    echo $sample_name

    seqkit seq 02_assembly/$sample_name/contigs.fasta -m 150 > 02_assembly/$sample_name/contigs_filt150.fasta

done




In [ ]:
mkdir 02_assembly_megahit

for sample_name in $(cat sample_for_binning.txt); do
    echo $sample_name

    megahit -o 02_assembly_megahit/$sample_name \
        -t 32 -m 0.3 \
        -1 ./01_adapter_removed/${sample_name}_1.fastq.gz -2 ./01_adapter_removed/${sample_name}_2.fastq.gz 

    # remove all sequences below 150bp because how can the assembled be shorter than the reads?
    # seqkit 2.9.0, cite paper
    #seqkit seq 02_assembly_megahit/$sample_name/final.contigs.fa -m 150 > 02_assembly_megahit/$sample_name/final.contigsfilt150.fa

done

conda deactivate
conda activate seqkit

for sample_name in $(cat ../sample_for_binning.txt); do
    echo $sample_name

    seqkit seq 02_assembly/$sample_name/contigs.fasta -m 300 > 02_assembly/$sample_name/contigs_filt300.fasta

done



In [ ]:
# mapping

mkdir 03_bowtie_metaspades

for sample_name in $(cat sample_for_binning.txt); do
    echo $sample_name

    bowtie2-build \
        --seed 42 --threads 32 \
        02_assembly/$sample_name/contigs_filt150.fasta 03_bowtie_metaspades/$sample_name \
        2>&1 | tee 03_bowtie_metaspades/log_build_$sample_name
    
    bowtie2 \
        -x 03_bowtie_metaspades/$sample_name \
        -1 01_adapter_removed/${sample_name}_1.fastq.gz \
        -2 01_adapter_removed/${sample_name}_2.fastq.gz \
        -S 03_bowtie_metaspades/${sample_name}.sam \
        --seed 42 --threads 32 \
        2>&1 | tee 03_bowtie_metaspades/log_align_$sample_name

done



for sample_name in $(cat ../sample_for_binning.txt); do
    echo $sample_name

    mapped=./03_bowtie/${sample_name}.sam
    bam_raw=./03_bowtie/${sample_name}_raw.bam
    bam_sorted=./03_bowtie/${sample_name}_sorted.bam
    bam_indexed=./03_bowtie/${sample_name}_sorted.bam.bai

    samtools view -o $bam_raw -F 4 -@ 32 -bS $mapped 
    samtools sort -o $bam_sorted -@ 32 $bam_raw 
    samtools index -b -@ 32 $bam_sorted $bam_indexed 

done


In [ ]:
bbmerge.sh in1= in2=<read2> out=<merged reads>

# merge pairs
parallel --xapply -j 2 \
    'bbmerge.sh in1={1} in2={2} out=./01_merged/{1/}' \
::: ./01_adapter_removed/*_1.fastq.gz ::: ./01_adapter_removed/*_2.fastq.gz

# 
parallel -j 4 \
    'seqtk seq -a {} > {= s:\.[^.]+$::;s:\.[^.]+$::; =}.fasta' \
::: ./01_merged/*_1.fastq.gz

docker run -it \
    -v /data/bioinformatics/databases/db-eggnog-mapper/:/data/bioinformatics/databases/db-eggnog-mapper/ \
    -v `pwd`:`pwd` -w `pwd` f01c4fb3b139

mkdir 08_annot_eggnog
EGGNOG_DATA_DIR=eggnog_db

parallel -j 4 \
    'emapper.py -i {} \
    --itype metagenome --genepred prodigal --cpu 32 \
    -m diamond --data_dir eggnog_db \
    --output {/.} --output_dir 08_annot_eggnog \
    --pident 80 --query_cover 50 --override' \
::: ./01_merged/*.fasta


for bin in $(find 01_merged/ -name "*.fasta" -type f); do
    file_name=$(basename $bin)
    bin_name=${file_name%.*}
    echo $bin_name
    emapper.py -i $bin \
    --itype metagenome --genepred prodigal --cpu 112 \
     -m diamond --data_dir $EGGNOG_DATA_DIR \
    --output $bin_name --output_dir 08_annot_eggnog --pident 80 --query_cover 50 --dbmem --override
done


seqtk sample -s100 01_adapter_removed/06_4A_1.fastq.gz 999 > sub_test1k.fastq.gz

emapper.py  -i sub_test99.fastq.gz --data_dir /data/bioinformatics/databases/db-eggnog-mapper/ -o test --itype metagenome -m diamond --cpu 32 --override

In [ ]:
mkdir 05_bins
mkdir 06_bin_deets

bowtie2-build \
    --seed 42 --threads 32 \
    05_bins_all.fa 06_bin_deets/bins

for F in ./01_adapter_removed/*_1.fastq.gz; do 
	R=${F%_*}_2.fastq.gz
	
	samplebase=$(basename "$F")
	samplename=${samplebase%_*}
    sam=./06_bin_deets/${samplename}.sam


	echo $samplename
    bowtie2 \
        -x 06_bin_deets/bins \
        -1 $F \
        -2 $R \
        -S $sam \
        --seed 42 --threads 128

    echo $samplename
    bam1_raw=./06_bin_deets/${samplename}_q2.bam
    bam1_sorted=./06_bin_deets/${samplename}_q2_sorted.bam
    bam1_index=./06_bin_deets/${samplename}_q2_sorted.bam.bai

    echo "Converting from sam to bam with MAPQ>=2..."    
    samtools view -o $bam1_raw -q 2 -F 4 -@ 32 -bS $sam
    
    echo "Sorting bam file..."
    samtools sort -o $bam1_sorted -@ 32 $bam1_raw 
    
    echo "Indexing bam file..."
    samtools index -b -@ 32 $bam1_sorted $bam1_index 
    
    echo "Removing $sam..."
    rm $sam
    
    echo "Done."
    echo "==========================================="

done





In [ ]:
#GTDBTK
